# <img style="float: left; padding-right: 20px; width: 200px" src="https://raw.githubusercontent.com/raxlab/imt2200-data/main/media/logo.jpg">  IMT 2200 - Introducción a Ciencia de Datos
**Pontificia Universidad Católica de Chile**<br>
**Instituto de Ingeniería Matemática y Computacional**<br>
**Profesor:** Rodrigo A. Carrasco <br>
---

# <h1><center>Clase 18: Exploración de Datos</center></h1>

En este notebook probaremos diferentes metodologías de exploración de datos sobre una base para generar hipótesis, visualizar información y entender qué información está en los datos.

## Caso de Estudio: Análisis de Votaciones Presidenciales en EEUU 2020.


### Descripción del Dataset.

Los datos para este problema correspoden a los resultados de las elecciones presidenciales 2020 de EEUU (Biden vs. Trump), a nivel de condados. Además, se incluyen algunas variables sociodemográficos y resultados de elecciones anteriores.

Estos datos pueden ser visualizados en el siguiente mapa de USA Today:

https://www.usatoday.com/in-depth/graphics/2020/11/10/election-maps-2020-america-county-results-more-voters/6226197002/


### Objetivo.

Analizar el comportamiento de distintos estados y condados en las elecciones presidenciales estadounidenses 2020. Por ejemplo, algunas preguntas que podríamos responder a partir de los datos son:

- ¿Cómo se distribuyen los condados donde ganó la opción demócrata (Biden) o republicana (Trump)?
- ¿En qué se parecen/diferencian?
- ¿Hay correlaciones entre el porcentaje de voto demócrata (o republicano) y otras variables políticas o sociodemográficas?
- ¿Es el comportamiento de un condado o estado, predictor de otro?
- ¿Hay similitudes entre los swing states?

## 1. Datos a analizar

El archivo `county_election_results.csv` será la base que usaremos para este trabajo.

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import seaborn as sns

### Lectura de Datos.

La librería Pandas implemente métodos de lectura de distintos tipos de datos, entre ellos datos en formato `csv`, mediante la función `read_csv`:

https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html

In [ ]:
dat = pd.read_csv('county_election_results.csv')

#caculamos el porcentaje de votos D y R en cada condado, para identificar más fácilmente quién ganó en cada uno
dat['pct_D'] = dat['votes_D_2020']/dat['votes_total_2020']*100
dat['pct_R'] = dat['votes_R_2020']/dat['votes_total_2020']*100

dat.head()

In [ ]:
dat.columns

In [ ]:
columns = ['pct_D','pct_R','acs_pct_pop_nonwhite', 'acs_pct_management', 'acs_pct_service',
       'pct_pop25+_bachelor_higher', 'median_household_income',
       'per_capita_income','POPESTIMATE2019','state','county','Unemployment Rate','winner','region']
dat2 = dat[columns]

## 2. EDA General

### 2.1 Revisemos algunas cosas generales de la base de datos

In [ ]:
dat.head()

In [ ]:
dat.info()

In [ ]:
dat[['Unemployment Rate','per_capita_income','acs_pct_pop_nonwhite']].describe()

In [ ]:
dat['votes_D_2020'].sum()+dat['votes_R_2020'].sum()+dat['votes_third_party_2020'].sum()

In [ ]:
dat['POPESTIMATE2019'].sum()

In [ ]:
100*(dat['votes_D_2020'].sum()+dat['votes_R_2020'].sum()+dat['votes_third_party_2020'].sum())/dat['POPESTIMATE2019'].sum()

### 2.2 Estadísticas de resumen


- **Media** (`np.mean()`): es la suma de todos los valores, dividida por el número de puntos.
 
$$\mu=\frac{1}{n} \sum_{i=1}^n x_i $$

- **Mediana** (`np.median()`): es el valor medio de un conjunto de datos. Es inmune a valores extremos o outliers. Para calcularla, se ordenan los datos y se elige el valor que queda en la mitad. 
 
- **Percentiles** (`np.percentile()`): el percentil $p$, corresponde al valor que es mayor al $p\%$ de los datos.


- **Varianza** (`np.var()`): promedio de la distancia cuadrática de los datos a la media. Es una medida de la dispersión de los datos.

$$\rm{Var}=\frac{1}{n} \sum_{i=1}^n (x_i-\bar{x})^2$$

- **Desviación estándar** (`np.std()`): es la raíz cuadrada de la varianza. Está en la misma escala de unidades que los datos.

$$\sigma=\sqrt{\frac{1}{n} \sum_{i=1}^n (x_i-\bar{x})^2}$$

- **Covarianza** (`np.cov()`): es una medida de cómo dos cantidades varían juntas.
Es la media del producto entre las diferencias de los valores respecto a la media.

$$\rm{Cov} (x,y)=\frac{1}{n} \sum_{i=1}^n (x_i-\bar{x})(y_i-\bar{y})$$

- **Coeficiente de correlación de Pearson** (`np.corrcoef`): para tener una medida más general y aplicable de la correlación entre dos variables, necesitamos que sea adimensional. Por lo tanto dividimos la covarianza por las desviaciones estándar de $x$ e $y$. Esto define el coeficiente de correlación de Pearson ($\rho$).

$$\rho=\frac{\rm{Cov} (x,y)}{\sigma_x \sigma_y}$$

Analicemos las principales estadísticas de resumen de los datos. ¿ Qué conclusiones útiles podemos obtener?

In [ ]:
dat['Unemployment Rate'].max()

In [ ]:
dat['Unemployment Rate'].std()

In [ ]:
np.percentile(dat['Unemployment Rate'],10)

In [ ]:
dat['winner'].value_counts()

In [ ]:
pd.pivot_table(dat,index='winner',values=['pct_pop25+_bachelor_higher'],aggfunc=['mean'])

### Algunas preguntas a responder:

- ¿Cuántos condados ganó cada candidato?
- ¿Hay diferencias en los ingresos per capita entre condados que votaron demócrata o republicano?
- ¿Hay diferencias en el nivel educacional entre condados que votaron demócrata o republicano?
- ¿Son homogéneos los estados D/R en cuanto a ingresos, desempleo, educación?

<div class="alert alert-block alert-info">
<b> Desafío:</b> Contesten las cuatro preguntas anteriores con los datos de la base usada.
    
<b>Los estudiantes que me envíen un Jupyter Notebook antes del viernes, contestando correctamente estas preguntas, tendrán +0.3 en la Interrogación 1.</b>
</div>

## 3. Análisis Exploratorio Univariado

Podemos utilizar múltiples funcionalidades de `matplotlib`y `seaborn` para realizar un análisis exploratorio gráfico de los datos.

`Funciones de interés:`

- https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hist.html
- https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.boxplot.html

In [ ]:
# histograma
fig = plt.figure()
ax = fig.add_subplot(111)

var = 'Unemployment Rate'
#bins=np.arange(0,100,10)
n,bins,patches = ax.hist(dat[var], histtype='bar', ec='darkblue', fc='lightblue', bins=np.arange(0,30,2))
ax.set_xlabel(var)
ax.set_ylabel('N')

**Binning bias.**

Lo histogramas son informativos y permiten entender las características genelares de nuestros datos, pero la gran desventaja, es que los mismos datos pueden interpretarse de distinta forma dependiendo de la elección de bins. La elección del número de bins es arbitraria, lo cual puede generar sesgos (binning bias). Adicionalmente, en un histograma no estamos graficando todos los datos, sino que los agrupamos en bins, y por lo tanto perdemos sus valores. Una regla común es usar un número de bins igual a la raíz cuadrada del número de muestras.

In [ ]:
#Notar las diferencias de binning
fig=plt.figure(figsize=(10,5))
ax1=fig.add_subplot(121)
ax2=fig.add_subplot(122)

bins = np.arange(0,100,10)
nbins = int(np.sqrt(len(dat)))
print(nbins)

_=ax1.hist(dat['pct_D'], bins = bins, histtype='bar', ec='darkblue', fc='lightblue')
_=ax2.hist(dat['pct_D'], bins = nbins, histtype='bar', ec='darkblue', fc='lightblue')

for ax in [ax1,ax2]:
    ax.set_xlabel('Porcentaje de voto demócrata')
    ax.set_ylabel('Número de Condados')
    ax.axvline(x=50, ls=':', color='gray')
    ax.set_ylim(0,900)
    ax.set_xlim(0,100)

## 4. Análisis Exploratorio Multivariado

¿Cómo se comportan las distintas variables políticas y socioeconómicas por región, estado, o según el partido ganador? ¿Podemos sacar alguna conclusión a partir de distintos análisis gráficos?

In [ ]:
sns.histplot(data=dat2, x='pct_D', hue='winner')

In [ ]:
sns.relplot(data=dat2, x='pct_D', y='winner')

In [ ]:
dat2['winner'] = ['D' if x>=50.0 else 'R' for x in dat2['pct_D']]
sns.histplot(data=dat2,x='pct_D',hue='winner')

In [ ]:
sns.relplot(data=dat2, x='pct_D', y='winner')

In [ ]:
columns=['pct_D','pct_R','acs_pct_pop_nonwhite', 'acs_pct_management', 'acs_pct_service',
       'pct_pop25+_bachelor_higher', 'median_household_income',
       'per_capita_income','POPESTIMATE2019','state','county','Unemployment Rate','winner','region']### Boxplot multivariado.

In [ ]:
sns.boxplot(data=dat2, y='acs_pct_pop_nonwhite', x='winner', hue='region')

In [ ]:
fig = plt.figure(figsize=(20,5))
ax = fig.add_subplot(111)
sns.boxplot(ax=ax,data=dat2,y='acs_pct_pop_nonwhite',x='state',width=0.8)

In [ ]:
#analicemos ahora los coeficientes de correlación para un subconjunto de condados: los que están en swing states
swing = ['GA','PA','AZ','MI','WI',]
dats = dat2[dat2['state'].isin(swing)]
dats

In [ ]:
sns.boxplot(data=dats,y='acs_pct_pop_nonwhite',x='winner',hue='state')

In [ ]:
fig = plt.figure(figsize=(20,5))
ax = fig.add_subplot(111)
sns.boxplot(data=dats,y='acs_pct_pop_nonwhite',x='state',hue='winner')

In [ ]:
sns.boxplot(data=dat2,y='acs_pct_pop_nonwhite',x='region',hue='winner')

In [ ]:
sns.boxplot(data=dats,y='acs_pct_pop_nonwhite',x='state',hue='winner')

In [ ]:
sns.boxplot(data=dats,y='Unemployment Rate',x='state',hue='winner')

Para los swing states, se aprecia una tendencia entre el ganador del condado, y el porcentaje de población no-blanca: en general este porcentaje tiene una distribución de valores más altos en los condados donde ganan los demócratas.

### 4.1 Gráficos de dispersión.

Podemos visualizar en más detalle la relación entre estas (y otras variables) usando gráficos de dispersión.

In [ ]:
sns.pairplot(dat2)

In [ ]:
sns.pairplot(dat2,y_vars='pct_D')

In [ ]:
sns.pairplot(dats,y_vars=['pct_D'],x_vars=['acs_pct_pop_nonwhite', 'acs_pct_management', 'Unemployment Rate'],hue='state',aspect=1);

### 4.2 Coeficientes de correlación.

Veamos ahora cómo es la correlación entre variables y cómo visualizarla

In [ ]:
#para calcular los coeficientes de correlación entre todas las variables, es más eficiente usar al función .corr() de pandas
corr = dats[['pct_D', 'pct_R', 'acs_pct_pop_nonwhite', 'acs_pct_management', 'acs_pct_service', 'pct_pop25+_bachelor_higher', 'median_household_income', 'per_capita_income', 'POPESTIMATE2019', 'Unemployment Rate']].corr()
corr

In [ ]:
sns.heatmap(corr);